In [1]:
import pandas as pd
import cv2 as cv
import numpy as np
from scipy.stats import skew
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.preprocessing import MinMaxScaler


In [2]:
train_df = pd.read_csv("./dataset/splits/train.csv", index_col='index')
train_df.head()
test_df = pd.read_csv("./dataset/splits/test.csv", index_col='index')
test_df.head()

,clip_name,clip_path,label,encoded_label
index,,,,
0,v_Diving_g25_c04.avi,./dataset/Diving/v_Diving_g25_c04.avi,Diving,0
1,v_Diving_g23_c01.avi,./dataset/Diving/v_Diving_g23_c01.avi,Diving,0
2,v_Diving_g16_c02.avi,./dataset/Diving/v_Diving_g16_c02.avi,Diving,0
3,v_Diving_g08_c06.avi,./dataset/Diving/v_Diving_g08_c06.avi,Diving,0
4,v_Diving_g13_c01.avi,./dataset/Diving/v_Diving_g13_c01.avi,Diving,0


In [3]:
def manual_normalize(frame):
    mean, std = cv.meanStdDev(frame)
    mean = mean.flatten()
    std = std.flatten()
    return (frame - mean) / std

def open_cv_normalize(frame):
    return cv.normalize(frame, None, alpha=0, beta=1.0, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

def open_cv_normalization_255(frame):
    return cv.normalize(frame, None, alpha=0, beta=255,
                               norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)


In [4]:
def color_segmentation(frame):
    rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    hsv_frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    resized_rgb = cv.resize(rgb_frame, (224, 224), interpolation=cv.INTER_CUBIC)
    resized_hsv = cv.resize(hsv_frame, (224, 224), interpolation=cv.INTER_CUBIC)

    hist_features = {}
    for i, col in enumerate(['red', 'green', 'blue']):
        hist = cv.calcHist(resized_rgb, [i], None, [256], [0, 256])
        hist_features[col] = hist.flatten()

    moments = []
    for i in range(3):
        channel = resized_hsv[:, :, i]
        mean = np.mean(channel)
        std = np.std(channel)
        skewness = skew(channel.flatten())
        if np.isnan(skewness):
            skewness = 0
        moments.append([mean, std, skewness])

    avg_color = np.mean(resized_rgb, axis=(0, 1))
    return hist_features, moments, avg_color


In [5]:
def add_color_feature_pipeline(index, file_path) -> list:
    file = cv.VideoCapture(file_path)
    results = []
    while file.isOpened():
        if not file:
            print("Error opening video stream or file")
        ret, frame = file.read()

        if ret:
            hist_features, moments, avg_color = color_segmentation(frame)

            results.append({'index': index,
                'moments_mean_h': moments[0][0], 'moments_std_h': moments[0][1], 'moments_skew_h': moments[0][2],
                'moments_mean_s': moments[1][0], 'moments_std_s': moments[1][1], 'moments_skew_s': moments[1][2],
                'moments_mean_v': moments[2][0], 'moments_std_v': moments[2][1], 'moments_skew_v': moments[2][2],
                'avg_color_r': avg_color[0], 'avg_color_g': avg_color[1], 'avg_color_b': avg_color[2]})
        else:
            break
    file.release()
    return results

In [6]:
COLOR_FEATURE_COLUMNS = [
    "moments_mean_h", "moments_std_h", "moments_skew_h",
    "moments_mean_s", "moments_std_s", "moments_skew_s",
    "moments_mean_v", "moments_std_v", "moments_skew_v",
    "avg_color_r", "avg_color_g", "avg_color_b",
]


In [7]:

scaler = MinMaxScaler()


def process_df(df, is_test=False):
    features = []
    for index, row in df.iterrows():
        if index > 10:
            break
        print(f"The index {index} amd row={row}")
        features += add_color_feature_pipeline(index, row.clip_path)

    color_df = pd.DataFrame(
        columns=['index', 'moments_mean_h', 'moments_std_h', 'moments_skew_h', 'moments_mean_s', 'moments_std_s',
                 'moments_skew_s', 'moments_mean_v', 'moments_std_v', 'moments_skew_v', 'avg_color_r', 'avg_color_g',
                 'avg_color_b'],
        data=features)

    for f in COLOR_FEATURE_COLUMNS:
        color_df[f] = color_df[f].astype(float)
        if is_test:
            color_df[COLOR_FEATURE_COLUMNS] = scaler.transform(color_df[COLOR_FEATURE_COLUMNS])
        else:
            color_df[COLOR_FEATURE_COLUMNS] = scaler.fit_transform(color_df[COLOR_FEATURE_COLUMNS])
    return color_df


In [8]:
train_color_df = process_df(train_df)

The index 0 amd row=clip_name                         v_Diving_g03_c01.avi
clip_path        ./dataset/Diving/v_Diving_g03_c01.avi
label                                           Diving
encoded_label                                        0
Name: 0, dtype: object
The index 1 amd row=clip_name                         v_Diving_g19_c03.avi
clip_path        ./dataset/Diving/v_Diving_g19_c03.avi
label                                           Diving
encoded_label                                        0
Name: 1, dtype: object
The index 2 amd row=clip_name                         v_Diving_g03_c04.avi
clip_path        ./dataset/Diving/v_Diving_g03_c04.avi
label                                           Diving
encoded_label                                        0
Name: 2, dtype: object
The index 3 amd row=clip_name                         v_Diving_g05_c04.avi
clip_path        ./dataset/Diving/v_Diving_g05_c04.avi
label                                           Diving
encoded_label             

In [9]:
train_color_df['index'].value_counts()

index
5     218
6     216
9     212
2     197
0     190
1     171
4     143
8     134
3     130
10    110
7      98
Name: count, dtype: int64

In [10]:
train_df.head()

,clip_name,clip_path,label,encoded_label
index,,,,
0,v_Diving_g03_c01.avi,./dataset/Diving/v_Diving_g03_c01.avi,Diving,0
1,v_Diving_g19_c03.avi,./dataset/Diving/v_Diving_g19_c03.avi,Diving,0
2,v_Diving_g03_c04.avi,./dataset/Diving/v_Diving_g03_c04.avi,Diving,0
3,v_Diving_g05_c04.avi,./dataset/Diving/v_Diving_g05_c04.avi,Diving,0
4,v_Diving_g15_c03.avi,./dataset/Diving/v_Diving_g15_c03.avi,Diving,0


In [11]:
train_color_df.head()

,index,moments_mean_h,moments_std_h,moments_skew_h,moments_mean_s,moments_std_s,moments_skew_s,moments_mean_v,moments_std_v,moments_skew_v,avg_color_r,avg_color_g,avg_color_b
0,0,0.318873,0.903319,0.826466,0.105273,0.264574,0.761678,0.733788,0.908757,0.637101,0.831488,0.878651,0.717682
1,0,0.321403,0.905182,0.824714,0.099609,0.255213,0.766077,0.733156,0.898408,0.633930,0.833929,0.878377,0.717649
2,0,0.306223,0.896965,0.832521,0.094993,0.249301,0.774881,0.731319,0.897980,0.635478,0.835402,0.878279,0.716317
3,0,0.306223,0.896965,0.832521,0.094976,0.249290,0.774908,0.731348,0.897940,0.635498,0.835442,0.878312,0.716344
4,0,0.283901,0.881938,0.840971,0.089171,0.244124,0.784089,0.732094,0.903653,0.641118,0.840198,0.881533,0.717625


In [12]:
merged_df = pd.merge(train_df, train_color_df, on='index', how='inner')

In [14]:
merged_df.head(1000)

,index,clip_name,clip_path,label,encoded_label,moments_mean_h,moments_std_h,moments_skew_h,moments_mean_s,moments_std_s,moments_skew_s,moments_mean_v,moments_std_v,moments_skew_v,avg_color_r,avg_color_g,avg_color_b
0,0,v_Diving_g03_c01.avi,./dataset/Diving/v_Diving_g03_c01.avi,Diving,0,0.318873,0.903319,0.826466,0.105273,0.264574,0.761678,0.733788,0.908757,0.637101,0.831488,0.878651,0.717682
1,0,v_Diving_g03_c01.avi,./dataset/Diving/v_Diving_g03_c01.avi,Diving,0,0.321403,0.905182,0.824714,0.099609,0.255213,0.766077,0.733156,0.898408,0.633930,0.833929,0.878377,0.717649
2,0,v_Diving_g03_c01.avi,./dataset/Diving/v_Diving_g03_c01.avi,Diving,0,0.306223,0.896965,0.832521,0.094993,0.249301,0.774881,0.731319,0.897980,0.635478,0.835402,0.878279,0.716317
3,0,v_Diving_g03_c01.avi,./dataset/Diving/v_Diving_g03_c01.avi,Diving,0,0.306223,0.896965,0.832521,0.094976,0.249290,0.774908,0.731348,0.897940,0.635498,0.835442,0.878312,0.716344
4,0,v_Diving_g03_c01.avi,./dataset/Diving/v_Diving_g03_c01.avi,Diving,0,0.283901,0.881938,0.840971,0.089171,0.244124,0.784089,0.732094,0.903653,0.641118,0.840198,0.881533,0.717625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5,v_Diving_g14_c01.avi,./dataset/Diving/v_Diving_g14_c01.avi,Diving,0,0.813952,0.033730,0.315553,0.876459,0.686827,0.211727,0.925676,0.129398,0.162025,0.168404,0.666034,0.931861
996,5,v_Diving_g14_c01.avi,./dataset/Diving/v_Diving_g14_c01.avi,Diving,0,0.817896,0.036324,0.417669,0.863461,0.680307,0.216645,0.930798,0.131055,0.170933,0.182796,0.671986,0.936680
997,5,v_Diving_g14_c01.avi,./dataset/Diving/v_Diving_g14_c01.avi,Diving,0,0.816685,0.035535,0.390244,0.858491,0.666497,0.207554,0.931590,0.136250,0.161142,0.186809,0.674015,0.937487
998,5,v_Diving_g14_c01.avi,./dataset/Diving/v_Diving_g14_c01.avi,Diving,0,0.820051,0.029867,0.457235,0.857173,0.653062,0.204791,0.936779,0.130538,0.161923,0.191795,0.677633,0.942750


In [48]:
svm_model = SVC(kernel='rbf', C=1.0, random_state=42, probability=True)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)


def train_pipeline():
    train_color_df = process_df(train_df)
    merged_df = pd.merge(train_df, train_color_df, left_index=True, right_index=True, how='inner')
    merged_df = merged_df.drop(columns=['clip_path', 'clip_name', 'label'])
    output = merged_df['encoded_label']
    merged_df = merged_df.drop(columns=['encoded_label'])
    svm_model.fit(merged_df, output)
    rf_model.fit(merged_df, output)


train_pipeline()


The index 0 amd row=clip_name                         v_Diving_g03_c01.avi
clip_path        ./dataset/Diving/v_Diving_g03_c01.avi
label                                           Diving
encoded_label                                        0
Name: 0, dtype: object
The index 1 amd row=clip_name                         v_Diving_g19_c03.avi
clip_path        ./dataset/Diving/v_Diving_g19_c03.avi
label                                           Diving
encoded_label                                        0
Name: 1, dtype: object
The index 2 amd row=clip_name                         v_Diving_g03_c04.avi
clip_path        ./dataset/Diving/v_Diving_g03_c04.avi
label                                           Diving
encoded_label                                        0
Name: 2, dtype: object
The index 3 amd row=clip_name                         v_Diving_g05_c04.avi
clip_path        ./dataset/Diving/v_Diving_g05_c04.avi
label                                           Diving
encoded_label             

/tmp/ipykernel_206344/1771590084.py:17: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness = skew(channel.flatten())


The index 17 amd row=clip_name                         v_Diving_g23_c02.avi
clip_path        ./dataset/Diving/v_Diving_g23_c02.avi
label                                           Diving
encoded_label                                        0
Name: 17, dtype: object
The index 18 amd row=clip_name                         v_Diving_g20_c05.avi
clip_path        ./dataset/Diving/v_Diving_g20_c05.avi
label                                           Diving
encoded_label                                        0
Name: 18, dtype: object
The index 19 amd row=clip_name                         v_Diving_g02_c07.avi
clip_path        ./dataset/Diving/v_Diving_g02_c07.avi
label                                           Diving
encoded_label                                        0
Name: 19, dtype: object
The index 20 amd row=clip_name                         v_Diving_g01_c06.avi
clip_path        ./dataset/Diving/v_Diving_g01_c06.avi
label                                           Diving
encoded_label      

ValueError: setting an array element with a sequence.

In [14]:
test_color_df = process_df(test_df, is_test=True)
merged_df = pd.merge(test_df, test_color_df, left_index=True, right_index=True, how='inner')
expected_output = test_df['encoded_label']
# merged_df = merged_df.drop(columns=['clip_path', 'Unnamed: 0', 'clip_name', 'index', 'label', 'encoded_label'])
# svm_output = svm_model.predict(merged_df)
# rf_output = rf_model.predict(merged_df)
# print('SVM classification report:')
# print(classification_report(expected_output, svm_output))
# print('RF classification report:')
# print(classification_report(expected_output, rf_output))
#
# fig, ax = plt.subplots(1, 2, figsize=(12, 5))
# ConfusionMatrixDisplay.from_predictions(expected_output, svm_output, ax=ax[0], cmap='Blues')
# ConfusionMatrixDisplay.from_predictions(expected_output, svm_output, ax=ax[1], cmap='Greens')
# plt.show()
#
# rf_prob = rf_model.predict_proba(merged_df)
# fig, ax = plt.subplots(figsize=(12, 5))
# for i, class_label in enumerate(rf_model.classes_):
#     PrecisionRecallDisplay.from_predictions(expected_output == class_label, rf_prob[:, i], name=class_label, ax=ax)
# ax.set_title('Precision-Recall curve (One vs Rest)')
# plt.show()


In [17]:
all_index = merged_df['index'].unique()

In [23]:
merged_df['index']

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
Name: index, dtype: int64

In [21]:
for index in all_index:
    print(index)

0


In [25]:
merged_df.head(1000)

,Unnamed: 0,clip_name,clip_path,label,encoded_label,index,moments_mean_h,moments_std_h,moments_skew_h,moments_mean_s,moments_std_s,moments_skew_s,moments_mean_v,moments_std_v,moments_skew_v,avg_color_r,avg_color_g,avg_color_b
0,0,v_Diving_g15_c01.avi,./dataset/Diving/v_Diving_g15_c01.avi,Diving,0,0,34.978994,34.865057,1.239489,52.096201,35.605027,0.423675,73.109794,48.399606,0.736082,68.338110,68.376475,62.971421
1,1,v_Diving_g17_c01.avi,./dataset/Diving/v_Diving_g17_c01.avi,Diving,0,0,35.181382,35.519648,1.242380,51.676578,35.222393,0.373033,73.453404,48.350301,0.711101,68.567223,68.532227,63.444555
2,2,v_Diving_g18_c04.avi,./dataset/Diving/v_Diving_g18_c04.avi,Diving,0,0,35.729592,35.049115,1.208825,52.096819,35.205047,0.359204,73.477559,47.691873,0.664251,68.377910,68.798888,63.443200
3,3,v_Diving_g02_c07.avi,./dataset/Diving/v_Diving_g02_c07.avi,Diving,0,0,36.082370,35.961212,1.195579,51.760124,35.085476,0.341791,73.947066,47.608199,0.640423,68.648497,69.074637,64.046476
4,4,v_Diving_g24_c05.avi,./dataset/Diving/v_Diving_g24_c05.avi,Diving,0,0,36.628308,35.295786,1.123996,52.295699,35.125891,0.340131,74.203763,47.419478,0.616698,68.769352,69.497828,64.283103
5,5,v_Diving_g08_c05.avi,./dataset/Diving/v_Diving_g08_c05.avi,Diving,0,0,36.705576,35.756476,1.116658,52.658602,35.449702,0.359769,74.807438,47.503689,0.602266,68.971142,69.926459,64.962093
6,6,v_Diving_g09_c03.avi,./dataset/Diving/v_Diving_g09_c03.avi,Diving,0,0,37.187659,35.053060,1.050918,53.187978,35.711956,0.389040,74.847118,47.429801,0.599215,68.920719,70.227220,64.968750
7,7,v_Diving_g21_c06.avi,./dataset/Diving/v_Diving_g21_c06.avi,Diving,0,0,37.429349,35.883133,1.051392,52.790039,35.542386,0.378784,75.391004,47.646756,0.590051,69.276268,70.565410,65.675423
8,8,v_Diving_g21_c03.avi,./dataset/Diving/v_Diving_g21_c03.avi,Diving,0,0,38.405991,37.466117,1.076577,53.795938,35.679951,0.277625,76.208705,48.485874,0.595437,69.524932,71.004624,66.518136
9,0,v_Drumming_g15_c02.avi,./dataset/Drumming/v_Drumming_g15_c02.avi,Drumming,1,0,38.821728,38.994615,1.074534,52.986228,35.001130,0.235320,76.580018,47.998404,0.551676,69.871213,70.989756,67.025411


In [12]:
svm_output[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [10]:
# def test_pipeline():
#     test_color_df = process_df(test_df, is_test=True)
#     merged_df = pd.merge(test_df, test_color_df, left_index=True, right_index=True, how='inner')
#     expected_output = test_df['encoded_label']
#     merged_df = merged_df.drop(columns=['clip_path', 'Unnamed: 0', 'clip_name', 'index', 'label', 'encoded_label'])
#     svm_output = svm_model.predict(merged_df)
#     rf_output = rf_model.predict(merged_df)
#     print('SVM classification report:')
#     print(classification_report(expected_output, svm_output))
#     print('RF classification report:')
#     print(classification_report(expected_output, rf_output))
#
#     fig, ax = plt.subplots(1, 2, figsize=(12, 5))
#     ConfusionMatrixDisplay.from_predictions(expected_output, svm_output, ax=ax[0], cmap='Blues')
#     ConfusionMatrixDisplay.from_predictions(expected_output, svm_output, ax=ax[1], cmap='Greens')
#     plt.show()
#
#     rf_prob = rf_model.predict_proba(merged_df)
#     fig, ax = plt.subplots(figsize=(12, 5))
#     for i, class_label in enumerate(rf_model.classes_):
#         PrecisionRecallDisplay.from_predictions(expected_output == class_label, rf_prob[:, i], name=class_label, ax=ax)
#     ax.set_title('Precision-Recall curve (One vs Rest)')
#     plt.show()
#
#
# test_pipeline()
